In [ ]:
# Cell 1: Imports and Setup
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import defaultdict
from backend.config_loader import load_config
from backend.utils import load_clusters_from_hdf5, load_halo_traces_from_hdf5, get_cluster_trace_info, load_single_cluster_traces
from pymanticore.analysis.matplotlib import get_mplstyle_path, ManticoreColors

In [ ]:
# Cell 2: Configuration Loading
config = load_config()

print("Configuration loaded:")
print(f"Base directory: {config.global_config.basedir}")
print(f"Output directory: {config.global_config.output_dir}")
print(f"Observer coordinates: {config.global_config.observer_coords}")
print(f"Mode1 - eps: {config.mode1.eps}, min_samples: {config.mode1.min_samples}")
print(f"Mode2 - target_snapshot: {config.mode2.target_snapshot}, min_cluster_size: {config.mode2.min_cluster_size}")

mnras_style = "mnras"

# Investigate the `eps` and `min_samples` dependence

In [ ]:
# Cell: Load Cluster Data for Different Eps and Min_Samples Values
import numpy as np
import os
from backend.utils import load_clusters_from_hdf5

# Define parameter ranges
eps_values = [1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 5.5]
min_samples_values = [3, 5, 7]

# Storage for results
eps_min_samples_results = {}

for min_samples in min_samples_values:
    eps_min_samples_results[min_samples] = {}
    
    for eps in eps_values:
        # Convert eps to filename format (2.5 -> "2p5")
        eps_str = str(eps).replace('.', 'p')
        filename = f"clusters_eps_{eps_str}_min_samples_{min_samples}.h5"
        
        try:
            clusters, cluster_metadata = load_clusters_from_hdf5(config.global_config.output_dir, filename=filename)
            
            # Calculate metrics
            n_clusters = len(clusters)
            cluster_sizes = [c['cluster_size'] for c in clusters]
            mean_cluster_size = np.mean(cluster_sizes) if cluster_sizes else 0
            max_cluster_size = max(cluster_sizes) if cluster_sizes else 0
            large_clusters = len([s for s in cluster_sizes if s >= 25])
            
            eps_min_samples_results[min_samples][eps] = {
                'n_clusters': n_clusters,
                'mean_size': mean_cluster_size,
                'max_size': max_cluster_size,
                'large_clusters': large_clusters,
                'cluster_sizes': cluster_sizes,
                'clusters': clusters,
                'metadata': cluster_metadata
            }
            
            print(f"min_samples={min_samples}, eps={eps}: {n_clusters} clusters, mean size={mean_cluster_size:.1f}, max size={max_cluster_size}")
            
        except FileNotFoundError:
            print(f"File not found for min_samples={min_samples}, eps={eps}: {filename}")
            eps_min_samples_results[min_samples][eps] = None
        except Exception as e:
            print(f"Error loading min_samples={min_samples}, eps={eps}: {e}")
            eps_min_samples_results[min_samples][eps] = None

In [ ]:
# Cell: Plot Number of Clusters vs Eps for Different Min_Samples
import matplotlib.pyplot as plt

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

colors = ['blue', 'red', 'green', 'orange', 'purple']
markers = ['o', 's', '^', 'D', 'v']

for i, min_samples in enumerate(min_samples_values):
    # Extract data for this min_samples value
    valid_eps = []
    n_clusters_list = []
    mean_sizes = []
    large_clusters_list = []
    
    for eps in eps_values:
        if eps_min_samples_results[min_samples].get(eps) is not None:
            valid_eps.append(eps)
            n_clusters_list.append(eps_min_samples_results[min_samples][eps]['n_clusters'])
            mean_sizes.append(eps_min_samples_results[min_samples][eps]['mean_size'])
            large_clusters_list.append(eps_min_samples_results[min_samples][eps]['large_clusters'])
    
    if len(valid_eps) > 0:
        color = colors[i % len(colors)]
        marker = markers[i % len(markers)]
        
        # Plot 1: Total number of clusters vs eps
        ax1.plot(valid_eps, n_clusters_list, color=color, marker=marker, 
                linewidth=2, markersize=8, label=f'min_samples={min_samples}')
        
        # Plot 2: Mean cluster size vs eps
        ax2.plot(valid_eps, mean_sizes, color=color, marker=marker, 
                linewidth=2, markersize=8, label=f'min_samples={min_samples}')
        
        # Plot 3: Large clusters (size >= 25) vs eps
        ax3.plot(valid_eps, large_clusters_list, color=color, marker=marker, 
                linewidth=2, markersize=8, label=f'min_samples={min_samples}')

# Configure Plot 1
ax1.set_xlabel('Eps (Mpc)')
ax1.set_ylabel('Number of Clusters')
ax1.set_title('Total Clusters vs Eps')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Configure Plot 2
ax2.set_xlabel('Eps (Mpc)')
ax2.set_ylabel('Mean Cluster Size')
ax2.set_title('Mean Cluster Size vs Eps')
ax2.grid(True, alpha=0.3)
ax2.legend()

# Configure Plot 3
ax3.set_xlabel('Eps (Mpc)')
ax3.set_ylabel('Number of Large Clusters (≥25)')
ax3.set_title('Large Clusters vs Eps')
ax3.grid(True, alpha=0.3)
ax3.legend()

# Plot 4: Summary table as text
ax4.axis('off')
table_text = "Parameter Sweep Summary\n\n"
table_text += f"{'MinSamp':<8} {'Eps':<6} {'N_Clust':<8} {'Mean_Size':<10} {'Large_Clust':<12}\n"
table_text += "-" * 60 + "\n"

for min_samples in min_samples_values:
    for eps in eps_values:
        data = eps_min_samples_results[min_samples].get(eps)
        if data is not None:
            table_text += f"{min_samples:<8} {eps:<6} {data['n_clusters']:<8} {data['mean_size']:<10.1f} {data['large_clusters']:<12}\n"

ax4.text(0.05, 0.95, table_text, transform=ax4.transAxes, fontsize=10, 
         verticalalignment='top', fontfamily='monospace')

plt.tight_layout()
plt.show()

# Print summary table
print(f"{'MinSamp':<8} {'Eps':<6} {'N_Clusters':<10} {'Mean_Size':<10} {'Large_Clusters':<15}")
print("-" * 60)
for min_samples in min_samples_values:
    for eps in eps_values:
        data = eps_min_samples_results[min_samples].get(eps)
        if data is not None:
            print(f"{min_samples:<8} {eps:<6} {data['n_clusters']:<10} {data['mean_size']:<10.1f} {data['large_clusters']:<15}")

# Load haloes

In [ ]:
eps = 2.0
min_samples = 5

# Convert eps to filename format (2.5 -> "2p5")
eps_str = str(eps).replace('.', 'p')
filename = f"clusters_eps_{eps_str}_min_samples_{min_samples}.h5"

clusters, cluster_metadata = load_clusters_from_hdf5(config.global_config.output_dir, filename=filename,
                                                    minimal=False)
print(f"Successfully loaded {len(clusters)} clusters")
clusters_available = True

# Check for traces file existence without loading data
try:
    trace_filepath = os.path.join(config.global_config.output_dir, "halo_traces.h5")
    if os.path.exists(trace_filepath):
        print("Halo trace file found")
        traces_available = True
    else:
        print("No halo trace data found. Run Mode 2 for temporal evolution plots.")
        traces_available = False
except Exception:
    print("No halo trace data found. Run Mode 2 for temporal evolution plots.")
    traces_available = False

In [ ]:
# Cell 4: Data Summary Statistics
if clusters_available:
    cluster_sizes = [c['cluster_size'] for c in clusters]
    
    print("Cluster Statistics:")
    print(f"  Total clusters: {len(clusters)}")
    print(f"  Largest cluster size: {max(cluster_sizes) if cluster_sizes else 0}")
    print(f"  Mean cluster size: {np.mean(cluster_sizes):.2f}")
    print(f"  Median cluster size: {np.median(cluster_sizes):.2f}")
    
    print(f"\nTop 5 clusters by size:")
    sorted_clusters = sorted(clusters, key=lambda x: x['cluster_size'], reverse=True)
    for i, cluster in enumerate(sorted_clusters[:5]):
        print(f"  {i+1}. Cluster {cluster['cluster_id']}: {cluster['cluster_size']} members, "
              f"mass={cluster['mean_mass']:.2e}")

if traces_available:
    cluster_trace_counts = get_cluster_trace_info(config.global_config.output_dir)
    total_traced_haloes = sum(cluster_trace_counts.values())
    
    print(f"\nTrace Statistics:")
    print(f"  Total traced haloes: {total_traced_haloes}")
    print(f"  Clusters with traces: {len(cluster_trace_counts)}")

# Cluster sizes

## Cluster sizes in random region

In [ ]:
# empirical_null_analysis.py
import h5py
import matplotlib.pyplot as plt
import numpy as np

# Load cluster sizes from HDF5 file
with h5py.File('output/random_control_clusters.h5', 'r') as f:
    cluster_sizes = []
    clusters_grp = f['clusters']
    for cluster_name in clusters_grp.keys():
        cluster_sizes.append(clusters_grp[cluster_name].attrs['cluster_size'])

cluster_sizes = np.array(cluster_sizes)

# Empirical survival function: P(size >= n)
def empirical_survival(n):
    return np.mean(cluster_sizes >= n)

# Calculate significance thresholds
size_range = np.arange(3, max(cluster_sizes) + 1)
survival_probs = [empirical_survival(n) for n in size_range]

# Find thresholds for different significance levels
alpha_levels = [0.1, 0.05, 0.01, 0.001]
thresholds = {}

for alpha in alpha_levels:
    threshold_candidates = [n for n, p in zip(size_range, survival_probs) if p <= alpha]
    if threshold_candidates:
        thresholds[alpha] = min(threshold_candidates)
    else:
        thresholds[alpha] = None

# Create plots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. Empirical distribution (linear scale)
bins = np.arange(1, max(cluster_sizes) + 2) - 0.5
counts, _ = np.histogram(cluster_sizes, bins=bins, density=True)
bin_centers = np.arange(1, max(cluster_sizes) + 1)

ax1.bar(bin_centers, counts, alpha=0.7, edgecolor='black', width=0.8)
ax1.set_xlabel('Cluster Size')
ax1.set_ylabel('Probability Density')
ax1.set_title(f'Empirical Null Distribution (N={len(cluster_sizes)})')
ax1.grid(True, alpha=0.3)

# Add threshold lines
colors = ['orange', 'red', 'purple', 'black']
for i, (alpha, threshold) in enumerate(thresholds.items()):
    if threshold is not None:
        ax1.axvline(threshold, color=colors[i], linestyle='--', 
                   label=f'{(1-alpha)*100:.1f}% threshold: {threshold}')
ax1.legend()

# 2. Empirical distribution (log scale)
ax2.bar(bin_centers, counts, alpha=0.7, edgecolor='black', width=0.8)
ax2.set_xlabel('Cluster Size')
ax2.set_ylabel('Probability Density')
ax2.set_title('Empirical Null Distribution (Log Scale)')
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3)

# Add threshold lines
for i, (alpha, threshold) in enumerate(thresholds.items()):
    if threshold is not None:
        ax2.axvline(threshold, color=colors[i], linestyle='--', 
                   label=f'{(1-alpha)*100:.1f}% threshold: {threshold}')
ax2.legend()

# 3. Survival function (linear scale)
ax3.plot(size_range, survival_probs, 'bo-', markersize=4, linewidth=2)
ax3.set_xlabel('Cluster Size')
ax3.set_ylabel('P(Size ≥ n)')
ax3.set_title('Empirical Survival Function')
ax3.grid(True, alpha=0.3)

# Add significance levels
for i, alpha in enumerate(alpha_levels):
    ax3.axhline(alpha, color=colors[i], linestyle='--', alpha=0.7,
               label=f'α = {alpha}')
    if thresholds[alpha] is not None:
        ax3.plot(thresholds[alpha], alpha, 'o', color=colors[i], markersize=8)

ax3.legend()
ax3.set_ylim(0, 1)

# 4. Survival function (log scale)
ax4.plot(size_range, survival_probs, 'bo-', markersize=4, linewidth=2)
ax4.set_xlabel('Cluster Size')
ax4.set_ylabel('P(Size ≥ n)')
ax4.set_title('Empirical Survival Function (Log Scale)')
ax4.set_yscale('log')
ax4.grid(True, alpha=0.3)

# Add significance levels
for i, alpha in enumerate(alpha_levels):
    ax4.axhline(alpha, color=colors[i], linestyle='--', alpha=0.7,
               label=f'α = {alpha}')
    if thresholds[alpha] is not None:
        ax4.plot(thresholds[alpha], alpha, 'o', color=colors[i], markersize=8)

ax4.legend()

plt.tight_layout()
plt.show()

# Print summary statistics
print(f"Null Distribution Summary:")
print(f"Total clusters: {len(cluster_sizes)}")
print(f"Size range: {min(cluster_sizes)} - {max(cluster_sizes)}")
print(f"Mean size: {np.mean(cluster_sizes):.2f}")
print(f"Median size: {np.median(cluster_sizes):.2f}")

print(f"\nSignificance Thresholds:")
for alpha in alpha_levels:
    if thresholds[alpha] is not None:
        confidence = (1 - alpha) * 100
        n_expected = empirical_survival(thresholds[alpha]) * len(cluster_sizes)
        print(f"{confidence:5.1f}% confidence (α={alpha:5.3f}): size ≥ {thresholds[alpha]:2d} "
              f"(p={empirical_survival(thresholds[alpha]):.4f}, {n_expected:.1f} expected)")
    else:
        print(f"{(1-alpha)*100:5.1f}% confidence (α={alpha:5.3f}): No threshold found")

# Detailed tail probabilities
print(f"\nTail Probabilities:")
for size in range(10, min(30, max(cluster_sizes) + 1), 5):
    prob = empirical_survival(size)
    expected = prob * len(cluster_sizes)
    print(f"P(size ≥ {size:2d}) = {prob:.4f} ({expected:.1f} expected)")

## Cluster sizes from constrained region

In [ ]:
# Cell: Cluster Size Distribution
if clusters_available and len(clusters) > 0:
    cluster_sizes = [cluster['cluster_size'] for cluster in clusters]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Linear scale histogram
    ax1.hist(cluster_sizes, bins=range(1, max(cluster_sizes) + 2), alpha=0.7, color='steelblue', edgecolor='black', linewidth=0.5)
    ax1.set_xlabel('Cluster Size', fontsize=12)
    ax1.set_ylabel('Frequency', fontsize=12)
    ax1.set_title(f'Cluster Size Distribution (Linear)\nTotal Clusters: {len(clusters)}', fontsize=14)
    ax1.grid(True, alpha=0.3)
    
    # Log scale histogram
    ax2.hist(cluster_sizes, bins=range(1, max(cluster_sizes) + 2), alpha=0.7, color='steelblue', edgecolor='black', linewidth=0.5)
    ax2.set_xlabel('Cluster Size', fontsize=12)
    ax2.set_ylabel('Frequency', fontsize=12)
    ax2.set_title('Cluster Size Distribution (Log Scale)', fontsize=14)
    ax2.set_yscale('log')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Cluster size statistics:")
    print(f"  Total clusters: {len(cluster_sizes)}")
    print(f"  Mean size: {np.mean(cluster_sizes):.2f}")
    print(f"  Median size: {np.median(cluster_sizes):.2f}")
    print(f"  Min size: {min(cluster_sizes)}")
    print(f"  Max size: {max(cluster_sizes)}")
    print(f"  Standard deviation: {np.std(cluster_sizes):.2f}")

# Plot cluster locations (overview)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

min_members_plot = 40

if clusters_available:
    with plt.style.context(get_mplstyle_path(mnras_style)):
        significant_clusters = [h for h in clusters if h['cluster_size'] >= min_members_plot]
        if len(significant_clusters) == 0:
            print(f"No clusters found with at least {min_members_plot} members")
        else:
            # ---- FIRST FIGURE: 2D XY and Mollweide projections ----
            fig = plt.figure(figsize=(6.5, 3.15))
            gs = fig.add_gridspec(1, 2, width_ratios=[1, 2])

            ax1 = fig.add_subplot(gs[0, 0])
            ax2 = fig.add_subplot(gs[0, 1], projection='mollweide')

            tab20_colors = plt.cm.tab20(np.linspace(0, 1, 20))
            random.seed(42)
            assigned_colors = random.choices(tab20_colors, k=len(significant_clusters))

            for i, cluster in enumerate(significant_clusters):
                positions = cluster['member_data']['BoundSubhalo/CentreOfMass']
                color = assigned_colors[i]
                ax1.scatter(positions[:, 0], positions[:, 1], c=[color], s=1, alpha=0.7, marker=".")

                # Mollweide expects longitude (ra) in [-pi, pi] and latitude (dec) in radians
                ra = np.asarray(cluster['member_data']['ra'])
                dec = np.asarray(cluster['member_data']['dec'])
                ra_wrapped = np.where(ra > 180, ra - 360, ra)  # wrap to [-180, 180]
                ra_rad = np.radians(ra_wrapped)
                dec_rad = np.radians(dec)
                ax2.scatter(ra_rad, dec_rad, c=[color], s=1, alpha=0.7, marker=".")

            # 2D plot formatting
            ax1.set_xlabel('X (Mpc)')
            ax1.set_ylabel('Y (Mpc)')
            ax1.set_ylim(0,1000)
            ax1.set_xlim(0,1000)
            circle = plt.Circle((500, 500), 300, fill=False, color='black', linestyle='--', linewidth=2)
            ax1.add_patch(circle)
            ax1.set_aspect('equal')

            # Mollweide formatting
            ax2.grid(True, alpha=0.3)
            # Set xticks in radians for Mollweide, with labels in degrees
            tick_labels = np.arange(-150, 181, 30)
            ax2.set_xticks(np.radians(tick_labels))
            ax2.set_xticklabels([f"{tl}°" for tl in tick_labels])
            ax2.set_xlabel('Right Ascension [deg]')
            ax2.set_ylabel('Declination [deg]')

            plt.tight_layout(pad=0.1)
            plt.savefig("./plots/posterior_positions.pdf")
            plt.show()
            
            # ---- SECOND FIGURE: Histogram of Median Distances ----
            significant_median_distances = [
                np.median(cluster['member_data']['dist'])
                for cluster in significant_clusters
            ]
            fig, ax = plt.subplots(figsize=(3.15, 2.5))
            n, bins, patches = ax.hist(significant_median_distances, bins=np.arange(0,510,10),
                                       alpha=0.7, edgecolor='black')
            bin_centers = (bins[:-1] + bins[1:]) / 2
            r_squared_line = bin_centers**2
            scaling_factor = 1./900
            r_squared_line *= scaling_factor
            ax.plot(bin_centers, r_squared_line, 'r--', linewidth=2, label=r'$N \propto R^2$')
            ax.set_xlabel('Median Distance (Mpc)')
            ax.set_ylabel('Number of Clusters')
            ax.set_ylim(0, 1.1 * np.max(n))
            ax.legend()
            plt.tight_layout(pad=0.1)
            plt.savefig("./plots/median_distances.pdf")
            plt.show()


# Helper plot functions

In [ ]:
# Cell 6: Cluster Diagnostic Function
def plot_cluster_diagnostic(cluster_id=None):
    if not clusters_available or len(clusters) == 0:
        print("No clusters available for diagnostic plot")
        return
    
    # Find target cluster
    if cluster_id is None:
        target_cluster = max(clusters, key=lambda x: x['cluster_size'])
        cluster_type = "Largest"
    else:
        target_cluster = None
        for cluster in clusters:
            if cluster['cluster_id'] == cluster_id:
                target_cluster = cluster
                break
        
        if target_cluster is None:
            print(f"Cluster ID {cluster_id} not found")
            return
        cluster_type = "Selected"
    
    target_cluster_center = target_cluster['mean_position']
    target_cluster_id = target_cluster['cluster_id']
    
    all_positions = []
    all_cluster_ids = []
    
    for cluster in clusters:
        positions = cluster['member_data']['BoundSubhalo/CentreOfMass']
        cluster_id_val = cluster['cluster_id']
        all_positions.extend(positions)
        all_cluster_ids.extend([cluster_id_val] * len(positions))
    
    all_positions = np.array(all_positions)
    all_cluster_ids = np.array(all_cluster_ids)
    
    distances = np.linalg.norm(all_positions - target_cluster_center, axis=1)
    within_15mpc = distances <= 15.0
    
    nearby_positions = all_positions[within_15mpc]
    nearby_cluster_labels = all_cluster_ids[within_15mpc]
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    unique_labels = np.unique(nearby_cluster_labels)
    colors = plt.cm.tab10(np.linspace(0, 1, len(unique_labels)))
    
    projections = [
        (0, 1, 'X', 'Y', 'X-Y'),
        (0, 2, 'X', 'Z', 'X-Z'), 
        (1, 2, 'Y', 'Z', 'Y-Z')
    ]
    
    for ax_idx, (dim1, dim2, label1, label2, proj_name) in enumerate(projections):
        ax = axes[ax_idx]
        
        for i, label in enumerate(unique_labels):
            mask = nearby_cluster_labels == label
            if label == target_cluster_id:
                ax.scatter(nearby_positions[mask, dim1], nearby_positions[mask, dim2], 
                          c='red', s=80, alpha=0.8, label=f'{cluster_type} Cluster (ID {label})', edgecolors='darkred')
            else:
                ax.scatter(nearby_positions[mask, dim1], nearby_positions[mask, dim2], 
                          c=[colors[i]], s=40, alpha=0.6, label=f'Cluster {label}')
        
        ax.scatter(target_cluster_center[dim1], target_cluster_center[dim2], 
                  c='black', s=300, marker='*', label='Cluster Center', edgecolors='white', linewidth=2)
        
        circle = plt.Circle((target_cluster_center[dim1], target_cluster_center[dim2]), 
                           7.5, fill=False, color='black', linestyle='--', linewidth=2, 
                           label='7.5 Mpc (eps threshold)')
        ax.add_patch(circle)
        
        ax.set_xlabel(f'{label1} (Mpc)')
        ax.set_ylabel(f'{label2} (Mpc)')
        ax.set_title(f'{proj_name} projection')
        ax.set_aspect('equal')
        
        if ax_idx == 0:
            ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    fig.suptitle(f'Haloes within 15 Mpc of {cluster_type} Cluster\n'
                f'Cluster Size: {target_cluster["cluster_size"]}, ID: {target_cluster_id}', 
                fontsize=14)
    
    plt.tight_layout()
    plt.show()
    
    print(f"{cluster_type} cluster analysis:")
    print(f"  Cluster ID: {target_cluster_id}")
    print(f"  Size: {target_cluster['cluster_size']} members")
    print(f"  Center: [{target_cluster_center[0]:.1f}, {target_cluster_center[1]:.1f}, {target_cluster_center[2]:.1f}]")
    print(f"  Mean mass: {target_cluster['mean_mass']:.2e}")
    print(f"  Mass std: {target_cluster['mass_std']:.2e}")

In [ ]:
# Cell 8a: Single Cluster Trajectory Plot Function
def plot_cluster_trajectory(cluster_id, ax=None):
    traces = load_single_cluster_traces(cluster_id, config.global_config.output_dir)
    
    if traces is None:
        print(f"No traces available for cluster {cluster_id}")
        return None
    
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(12, 12))
        standalone = True
    else:
        standalone = False
    
    # Calculate cluster centroid from final positions (snapshot 77)
    final_positions = []
    for trace_data in traces:
        positions = trace_data['BoundSubhalo/CentreOfMass']
        snapshots = trace_data['snapshots']
        final_idx = np.where(snapshots == 77)[0]
        if len(final_idx) > 0:
            final_positions.append(positions[final_idx[0]])
    
    if len(final_positions) > 0:
        final_positions = np.array(final_positions)
        cluster_centroid = np.mean(final_positions, axis=0)
    else:
        cluster_centroid = np.array([0, 0, 0])
    
    # Generate colors for each trajectory
    colors = plt.cm.tab20(np.linspace(0, 1, min(20, len(traces))))
    if len(traces) > 20:
        colors = plt.cm.gist_ncar(np.linspace(0, 1, len(traces)))
    
    for i, trace_data in enumerate(traces):
        positions = trace_data['BoundSubhalo/CentreOfMass']
        snapshots = trace_data['snapshots']
        
        # Plot trajectory line
        ax.plot(positions[:, 0], positions[:, 1], '-', 
               alpha=0.5, linewidth=0.8, color=colors[i])
        
        # Plot start and end points
        ax.scatter(positions[0, 0], positions[0, 1], 
                  c='red', s=40, marker='s', alpha=0.9, 
                  edgecolors='darkred', linewidth=0.3, zorder=10)
        ax.scatter(positions[-1, 0], positions[-1, 1], 
                  c='blue', s=40, marker='o', alpha=0.9,
                  edgecolors='darkblue', linewidth=0.3, zorder=10)
    
    # Add radial rings at 10 and 20 Mpc
    circle_10 = plt.Circle((cluster_centroid[0], cluster_centroid[1]), 
                          10, fill=False, color='gray', linestyle='--', 
                          linewidth=1, alpha=0.7)
    circle_20 = plt.Circle((cluster_centroid[0], cluster_centroid[1]), 
                          20, fill=False, color='gray', linestyle=':', 
                          linewidth=1, alpha=0.7)
    ax.add_patch(circle_10)
    ax.add_patch(circle_20)
    
    # Set axis limits to +/- 15 Mpc from cluster centroid
    ax.set_xlim(cluster_centroid[0] - 15, cluster_centroid[0] + 15)
    ax.set_ylim(cluster_centroid[1] - 15, cluster_centroid[1] + 15)
    
    ax.set_xlabel('X (Mpc)', fontsize=10)
    ax.set_ylabel('Y (Mpc)', fontsize=10)
    ax.set_title(f'Cluster {cluster_id} (n={len(traces)})', fontsize=12, pad=10)
    ax.set_aspect('equal')
    ax.set_facecolor('#f8f8f8')
    ax.grid(True, alpha=0.2, linewidth=0.5)
    ax.tick_params(labelsize=8)
    
    if standalone:
        plt.tight_layout()
        plt.show()
    
    return traces

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_cluster_mass_evolution(cluster_id, ax=None):
    """
    Plot the median and 10th–90th percentile range of BoundSubhalo/TotalMass
    as a function of snapshot for all haloes in the given cluster.
    
    Parameters
    ----------
    cluster_id : int or str
        Identifier of the cluster to load.
    ax : matplotlib.axes.Axes, optional
        Axes in which to plot. If None, creates a new figure + axes.

    Returns
    -------
    None
    """
    # Load all traces for this cluster
    traces = load_single_cluster_traces(cluster_id, config.global_config.output_dir)
    if traces is None or len(traces) == 0:
        print(f"No traces available for cluster {cluster_id}")
        return

    # Gather all unique snapshot indices present
    all_snaps = np.unique(np.concatenate([t['snapshots'] for t in traces]))
    all_snaps.sort()

    # For each snapshot, collect the masses from each trace (where available)
    medians = []
    p10 = []
    p90 = []

    for snap in all_snaps:
        masses_at_snap = []
        for t in traces:
            # find if this trace has the snapshot
            idx = np.where(t['snapshots'] == snap)[0]
            if idx.size > 0:
                masses_at_snap.append(t['BoundSubhalo/TotalMass'][idx[0]])
        if len(masses_at_snap) > 0:
            arr = np.array(masses_at_snap)
            p10.append(np.percentile(arr, 10))
            medians.append(np.percentile(arr, 50))
            p90.append(np.percentile(arr, 90))
        else:
            # no data for this snapshot
            p10.append(np.nan)
            medians.append(np.nan)
            p90.append(np.nan)

    # Prepare the plot
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 6))
        standalone = True
    else:
        standalone = False

    # Convert lists to arrays
    all_snaps = np.array(all_snaps)
    p10 = np.array(p10)
    medians = np.array(medians)
    p90 = np.array(p90)

    # Plot median line
    ax.plot(all_snaps, medians, '-', lw=2, label='Median Mass', color='C0')

    # Shade 10th–90th percentile range
    ax.fill_between(all_snaps, p10, p90, color='C0', alpha=0.3,
                    label='10th–90th percentile')

    # Labels & styling
    ax.set_xlabel('Snapshot', fontsize=12)
    ax.set_ylabel('BoundSubhalo / TotalMass', fontsize=12)
    ax.set_title(f'Cluster {cluster_id}: Mass Evolution (n={len(traces)})', pad=10)
    ax.grid(True, alpha=0.2)
    ax.legend(fontsize=10)
    ax.tick_params(labelsize=10)

    if standalone:
        plt.tight_layout()
        plt.show()


In [ ]:
# Cell: Single Cluster Mass Distribution Function
def plot_cluster_mass_distribution(cluster_id, ax=None):
    if not clusters_available:
        print(f"No clusters available")
        return
    
    # Find the cluster
    target_cluster = None
    for cluster in clusters:
        if cluster['cluster_id'] == cluster_id:
            target_cluster = cluster
            break
    
    if target_cluster is None:
        print(f"Cluster ID {cluster_id} not found")
        return
    
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(10, 6))
        standalone = True
    else:
        standalone = False
    
    cluster_masses = target_cluster['member_data']['BoundSubhalo/TotalMass']
    cluster_size = target_cluster['cluster_size']
    median_mass = np.median(cluster_masses)
    mean_mass = target_cluster['mean_mass']
    
    # Plot histogram of masses
    ax.hist(cluster_masses, bins=10**np.arange(13.8,15.5,0.05), alpha=0.7, color='skyblue', edgecolor='black', linewidth=0.5)
    
    # Add vertical lines for mean and median
    ax.axvline(mean_mass, color='red', linestyle='--', linewidth=2, alpha=0.8, label=f'Mean: {mean_mass:.2e}')
    ax.axvline(median_mass, color='orange', linestyle='-', linewidth=2, alpha=0.8, label=f'Median: {median_mass:.2e}')
    
    ax.set_xlabel('Mass (M☉)', fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)
    ax.set_title(f'Cluster {cluster_id} (n={cluster_size})', fontsize=12, pad=10)
    ax.tick_params(labelsize=8)
    ax.grid(True, alpha=0.3, linewidth=0.5)
    ax.set_xscale("log")
    ax.legend(fontsize=8)
    
    if standalone:
        plt.tight_layout()
        plt.show()

# Cluster evolutions

In [ ]:
# Cell 7: Temporal Evolution Setup
if traces_available:
    cluster_trace_counts = get_cluster_trace_info(config.global_config.output_dir)
    
    # Filter clusters with sufficient traces
    min_traces_for_plot = 5  # Adjustable parameter
    significant_trace_clusters = {k: v for k, v in cluster_trace_counts.items() 
                                 if v >= min_traces_for_plot}
    
    print(f"Temporal evolution setup:")
    print(f"  Total clusters with traces: {len(cluster_trace_counts)}")
    print(f"  Clusters with >= {min_traces_for_plot} traces: {len(significant_trace_clusters)}")
    
    # Sort by number of traces
    sorted_trace_clusters = sorted(significant_trace_clusters.items(), 
                                  key=lambda x: x[1], reverse=True)
    
    print(f"\nTop clusters by trace count:")
    for cluster_id, trace_count in sorted_trace_clusters[:10]:
        print(f"  Cluster {cluster_id}: {trace_count} traces")

In [ ]:
# Cell 8b: Top 9 Cluster Trajectory Plots (3x3 Grid)
if traces_available and len(significant_trace_clusters) > 0:
    # Plot top 9 clusters in 3x3 grid
    n_clusters_to_plot = min(9, len(sorted_trace_clusters))
    fig, axes = plt.subplots(3, 3, figsize=(20, 20))
    axes = axes.flatten()
    
    for plot_idx in range(n_clusters_to_plot):
        cluster_id, trace_count = sorted_trace_clusters[plot_idx]
        ax = axes[plot_idx]
        plot_cluster_trajectory(cluster_id, ax=ax)
    
    # Hide unused subplots
    for plot_idx in range(n_clusters_to_plot, 9):
        axes[plot_idx].set_visible(False)
    
    # Add overall legend
    fig.legend([plt.Line2D([0], [0], marker='s', color='red', linestyle='None', markersize=8, markeredgecolor='darkred'),
                plt.Line2D([0], [0], marker='o', color='blue', linestyle='None', markersize=8, markeredgecolor='darkblue'),
                plt.Line2D([0], [0], color='gray', alpha=0.6, linewidth=1.0),
                plt.Line2D([0], [0], color='gray', linestyle='--', linewidth=1, alpha=0.7),
                plt.Line2D([0], [0], color='gray', linestyle=':', linewidth=1, alpha=0.7)],
               [f'Snapshot 77 (start)', f'Snapshot {config.mode2.target_snapshot} (end)', 'Halo trajectories', '10 Mpc', '20 Mpc'],
               loc='upper center', bbox_to_anchor=(0.5, 0.02), ncol=5, fontsize=12)
    
    plt.suptitle(f'Top {n_clusters_to_plot} Clusters by Trace Count\nSnapshot {config.mode2.target_snapshot} to 77', 
                 fontsize=16, y=0.95)
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.05)
    plt.show()

# Mass distributions

In [ ]:
# Cell: Mass Distributions for Clusters Above Size Threshold (5xN Grid)
if clusters_available and len(clusters) > 0:
    # Set cluster size threshold
    cluster_size_threshold = 70  # Adjustable parameter
    
    # Filter clusters by size
    large_clusters = [cluster for cluster in clusters if cluster['cluster_size'] >= cluster_size_threshold]
    
    if len(large_clusters) == 0:
        print(f"No clusters found with size >= {cluster_size_threshold}")
    else:
        # Calculate grid dimensions (5 columns, as many rows as needed)
        n_cols = 5
        n_rows = (len(large_clusters) + n_cols - 1) // n_cols  # Ceiling division
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 5*n_rows))
        
        # Handle case where there's only one row
        if n_rows == 1:
            axes = axes.reshape(1, -1)
        axes = axes.flatten()
        
        # Sort by cluster size descending
        large_clusters.sort(key=lambda c: c['cluster_size'], reverse=True)
        
        for plot_idx in range(len(large_clusters)):
            cluster = large_clusters[plot_idx]
            ax = axes[plot_idx]
            plot_cluster_mass_distribution(cluster['cluster_id'], ax=ax)
        
        # Hide unused subplots
        for plot_idx in range(len(large_clusters), n_rows * n_cols):
            axes[plot_idx].set_visible(False)
        
        plt.suptitle(f'Mass Distributions for Clusters with Size >= {cluster_size_threshold}\n({len(large_clusters)} clusters)', fontsize=18, y=0.95)
        plt.tight_layout()
        #plt.subplots_adjust(top=0.92)
        plt.show()

# Inspect individual clusters

In [ ]:
cluster_id = 256
plot_cluster_diagnostic(cluster_id=cluster_id)
plot_cluster_mass_distribution(cluster_id=cluster_id)
_ = plot_cluster_trajectory(cluster_id=cluster_id)
plot_cluster_mass_evolution(cluster_id=cluster_id)

# Save median properties

In [ ]:
# Cell: Export Simplified Cluster Data
import h5py
import numpy as np
import os

# Configuration variables
min_cluster_size = 25  # Modify this threshold as needed
output_filename = "simplified_clusters.h5"  # Modify filename as needed

if clusters_available and len(clusters) > 0:
    # Filter clusters by size threshold
    filtered_clusters = [cluster for cluster in clusters if cluster['cluster_size'] >= min_cluster_size]
    
    if len(filtered_clusters) == 0:
        print(f"No clusters found with size >= {min_cluster_size}")
    else:
        print(f"Processing {len(filtered_clusters)} clusters with size >= {min_cluster_size}")
        
        # Get all property keys from first cluster
        property_keys = list(filtered_clusters[0]['member_data'].keys())
        
        # Initialize storage for median values
        cluster_ids = []
        cluster_sizes = []
        median_properties = {}
        
        # Initialize median property arrays
        for key in property_keys:
            sample_data = filtered_clusters[0]['member_data'][key]
            if len(sample_data.shape) == 1:
                # 1D property
                median_properties[key] = []
            elif len(sample_data.shape) == 2 and sample_data.shape[1] == 3:
                # 3D vector property
                median_properties[key] = []
            else:
                # Other multi-dimensional properties
                median_properties[key] = []
        
        # Calculate medians for each cluster
        for cluster in filtered_clusters:
            cluster_ids.append(cluster['cluster_id'])
            cluster_sizes.append(cluster['cluster_size'])
            
            for key in property_keys:
                data = cluster['member_data'][key]
                
                if len(data.shape) == 1:
                    # 1D property - simple median
                    median_val = np.nanmedian(data)
                    median_properties[key].append(median_val)
                elif len(data.shape) == 2 and data.shape[1] == 3:
                    # 3D vector - median per component
                    median_vec = np.nanmedian(data, axis=0)
                    median_properties[key].append(median_vec)
                else:
                    # Other shapes - median along first axis
                    median_val = np.nanmedian(data, axis=0)
                    median_properties[key].append(median_val)
        
        # Convert lists to arrays
        cluster_ids = np.array(cluster_ids)
        cluster_sizes = np.array(cluster_sizes)
        for key in property_keys:
            median_properties[key] = np.array(median_properties[key])
        
        # Save to HDF5
        output_path = os.path.join(config.global_config.output_dir, output_filename)
        
        with h5py.File(output_path, 'w') as f:
            # Metadata
            meta_grp = f.create_group('metadata')
            meta_grp.attrs['threshold_size'] = min_cluster_size
            meta_grp.attrs['n_clusters'] = len(filtered_clusters)
            meta_grp.attrs['source_file'] = 'clusters.h5'
            meta_grp.attrs['total_properties'] = len(property_keys)
            
            # Copy original metadata if available
            if 'cluster_metadata' in locals():
                for key, value in cluster_metadata.items():
                    if isinstance(value, (int, float, str)):
                        meta_grp.attrs[f'original_{key}'] = value
                    elif isinstance(value, (list, np.ndarray)):
                        meta_grp.attrs[f'original_{key}'] = np.array(value)
            
            # Cluster data
            clusters_grp = f.create_group('clusters')
            clusters_grp.create_dataset('cluster_ids', data=cluster_ids)
            clusters_grp.create_dataset('cluster_sizes', data=cluster_sizes)
            
            # Median properties
            median_grp = clusters_grp.create_group('median_properties')
            for key, data in median_properties.items():
                # Convert property name to HDF5-safe format
                dataset_name = key.replace('/', '_')
                median_grp.create_dataset(dataset_name, data=data)
        
        print(f"Saved simplified cluster data to: {output_path}")
        print(f"Properties included: {len(property_keys)}")
        print(f"Sample properties: {property_keys[:5]}{'...' if len(property_keys) > 5 else ''}")
        
        # Summary statistics
        print(f"\nSummary:")
        print(f"  Clusters processed: {len(filtered_clusters)}")
        print(f"  Size range: {np.min(cluster_sizes)} - {np.max(cluster_sizes)}")
        print(f"  Mean cluster size: {np.mean(cluster_sizes):.1f}")
        print(f"  File size: {os.path.getsize(output_path) / 1024**2:.2f} MB")

else:
    print("No cluster data available. Run Mode 1 first.")